In [1]:
# Xin quyen Google drive
from google.colab import drive
drive.mount('/content/ggdrive')

Mounted at /content/ggdrive


In [2]:
import pandas as pd
import numpy as np

In [3]:
# 1. Kiểm tra và chuẩn hóa tên cột

# Đọc file CSV
df = pd.read_csv("/content/ggdrive/MyDrive/Phân tích hiệu suất kinh doanh TIMA/Tima_CRM - Sheet1.csv")

# So sánh các cột
diff_initial = df[df['SoTienDKVayBanDau'] != df['Số tiền đăng ký vay ban đầu']]
diff_disbursed = df[df['TienGiaiNgan'] != df['Tiền giải ngân']]
diff_remaining = df[df['SoTienConLai'] != df['Tiền gốc còn lại']]

print("Khác biệt Số tiền đăng ký vay ban đầu:", diff_initial[['SoTienDKVayBanDau', 'Số tiền đăng ký vay ban đầu']])
print("Khác biệt Tiền giải ngân:", diff_disbursed[['TienGiaiNgan', 'Tiền giải ngân']])
print("Khác biệt Tiền gốc còn lại:", diff_remaining[['SoTienConLai', 'Tiền gốc còn lại']])

Khác biệt Số tiền đăng ký vay ban đầu:    SoTienDKVayBanDau  Số tiền đăng ký vay ban đầu
1                NaN                     10000000
Khác biệt Tiền giải ngân:    TienGiaiNgan  Tiền giải ngân
0           NaN        10000000
1      250000.0        10000000
Khác biệt Tiền gốc còn lại:       SoTienConLai  Tiền gốc còn lại
0                0          10000000
2           -50000           5000000
3                0          10000000
5                0          10000000
6                0           4000000
...            ...               ...
2374             0          13899322
2377      14254750          13441422
2378       3474831           1814424
2380             0           6000000
2381             0           6000000

[1674 rows x 2 columns]


In [4]:
# Xóa 2 cột SoTienDKVayBanDau và TienGiaiNgan
df = df.drop(['SoTienDKVayBanDau', 'TienGiaiNgan'], axis=1)

In [5]:
# 2.Xử lý lỗi định dạng(power querry)

In [6]:
# 3.Thống kê và xử lý dữ liệu thiếu

# Tính số lượng giá trị thiếu và phần trăm cho từng cột
missing_data = df.isnull().sum()
total_rows = len(df)
missing_percentage = (missing_data / total_rows * 100).round(2)

# Tạo DataFrame báo cáo
missing_report = pd.DataFrame({
    'Số lượng giá trị thiếu': missing_data,
    'Phần trăm thiếu (%)': missing_percentage
})

# Sắp xếp theo phần trăm thiếu giảm dần
missing_report = missing_report.sort_values(by='Phần trăm thiếu (%)', ascending=False)

# Lọc các cột có dữ liệu thiếu
missing_report = missing_report[missing_report['Số lượng giá trị thiếu'] > 0]

# In báo cáo
print("Báo cáo dữ liệu thiếu:")
print(missing_report)

Báo cáo dữ liệu thiếu:
                         Số lượng giá trị thiếu  Phần trăm thiếu (%)
ReceiveYourIncomeSalary                    1943                81.54
WardName                                   1716                72.01
CityCompany                                 610                25.60
DescriptionPositionJob                      482                20.23
FullNameFamily                              273                11.46
JobName                                     248                10.41
Address                                     218                 9.15
CreditInfo                                  217                 9.11
Name                                        217                 9.11
NameCompany                                 177                 7.43
Thời gian đã sống                           164                 6.88
AddressCompany                              161                 6.76
Hình thức cư trú                            142                 5.96
RelativeFam

In [7]:
#Xử lí dữ liệu thiếu
df['ReceiveYourIncomeSalary'] = df['ReceiveYourIncomeSalary'].fillna('Unknown')
df['WardName'] = df['WardName'].fillna('Unknown')
df['CityCompany'] = df['CityCompany'].fillna(df['CityName'])
df['DescriptionPositionJob'] = df['DescriptionPositionJob'].fillna('Unknown')
df['FullNameFamily'] = df['FullNameFamily'].fillna('Unknown')
df['JobName'] = df['JobName'].fillna('Unknown')
df['Address'] = df['Address'].fillna('Unknown')
df['CreditInfo'] = df['CreditInfo'].fillna('Unknown')
df['FullName'] = df['FullName'].replace('12345', 'Nguyễn Anh Vũ')
df = df.drop('Name', axis=1)
df['Thời gian đã sống'] = df['Thời gian đã sống'].fillna('Unknown')
df['AddressCompany'] = df['AddressCompany'].fillna('Unknown')
df['Hình thức cư trú'] = df['Hình thức cư trú'].fillna('Unknown')
df['RelativeFamilyName'] = df['RelativeFamilyName'].fillna('Unknown')
df['DistrictNameCompany'] = df['DistrictNameCompany'].fillna('Unknown')
df['DistrictNameCompany'] = df['DistrictNameCompany'].fillna(df['DistrictName'])
df['DistrictNameHouseHold'] = df['DistrictNameHouseHold'].fillna(df['DistrictName'])
df['CityNameHouseHold'] = df['CityNameHouseHold'].fillna(df['CityName'])
df['WardNameHouseHold'] = df['WardNameHouseHold'].fillna('Unknown')
df['Street'] = df['Street'].fillna('Unknown')
default_time = pd.Timestamp('2019-03-01 00:00:00')
df['CheckTime'] = df['CheckTime'].fillna(default_time)
df['Salary'] = df['Salary'].fillna('0')
df['ProductCreditName'] = df['ProductCreditName'].fillna('Unknown')
df['NameCompany'] = df['NameCompany'].fillna('Unknown')
df.loc[df['STT'] == 3094, 'Gender'] = '1'
df.loc[df['STT'] == 3116, 'Gender'] = '0'
df.loc[df['STT'] == 3117, 'Gender'] = '0'
df.loc[df['STT'] == 3182, 'Gender'] = '0'


In [8]:
# 4.Kiểm tra giá trị trùng lặp trên toàn bộ DataFrame
duplicates = df.duplicated()
print("Các hàng trùng lặp:\n", duplicates)

# Đếm số lượng hàng trùng lặp
duplicate_count = duplicates.sum()
print("Số hàng trùng lặp:", duplicate_count)

Các hàng trùng lặp:
 0       False
1       False
2       False
3       False
4       False
        ...  
2378    False
2379    False
2380    False
2381    False
2382    False
Length: 2383, dtype: bool
Số hàng trùng lặp: 0


In [9]:
# 5.Xử lí các giá trị sai, không hợp lí
# Thay giá trị không hợp lệ
df.loc[df['application_date'] == '2023-02-30', 'application_date'] = '2016-06-24'

In [10]:
df['SoTienConLai'] = df['SoTienConLai'].replace(-50000, 0)

In [11]:
df['Gender'] = df['Gender'].replace('Male', '0')

In [12]:
df['Hình thức cư trú'] = df['Hình thức cư trú'].replace('-1', "Hơn 10 năm")

In [13]:
df['Salary'] = df['Salary'].replace('Ten thousand', '7700000')
df['Salary'] = df['Salary'].replace('1000$', '26000000')

In [14]:
df['NumberOfLoans'] = df['NumberOfLoans'].replace(-1, 0)

In [15]:
#Hợp lí hóa dữ liệu cột SoTienConLai, TrangThai, CreditInfo, ...
df['Trạng thái'] = df['Trạng thái'].replace('Đang vay xong', 'Kết thúc')
# Thay thế SoTienConLai bằng 0 cho các hàng được lọc
df.loc[(df['Trạng thái'] == 'Kết thúc') & (df['SoTienConLai'] != 0), 'SoTienConLai'] = 0
# Lọc các hàng có Trạng thái là Nợ Xấu và điền HasBadDebt thành 1
df.loc[df['Trạng thái'] == 'Nợ Xấu', 'HasBadDebt'] = 1
# Cập nhật cột SoTienConLai dựa trên điều kiện STT
df.loc[df['STT'].isin([609, 621, 637, 684, 840]), 'SoTienConLai'] = df['Tiền giải ngân']

In [16]:
df['Thời gian đã sống'] = df['Thời gian đã sống'].replace('-1', 'Hơn 10 năm')
df = df.drop('Tiền gốc còn lại', axis=1)
df = df.rename(columns={'SoTienConLai': 'Tiền gốc còn lại'})

In [17]:
# 6.chuẩn hóa SĐT
import re
def standardize_phone_number(phone):
    if pd.isna(phone) or phone == '':
        return np.nan
    phone = str(phone).strip().replace(' ', '').replace('-', '')

    if not phone.isdigit():
        return np.nan  # Trả về NaN nếu chứa ký tự không hợp lệ (như abc1234567)

    if len(phone) == 9:
        phone = '0' + phone

    if len(phone) == 10 and phone.startswith('0'):
        return phone

    return np.nan

# Áp dụng hàm chuẩn hóa cho cột 'Số điện thoại khách hàng'
df['Số điện thoại khách hàng'] = df['Số điện thoại khách hàng'].apply(standardize_phone_number)
df['Số điện thoại khách hàng'] = df['Số điện thoại khách hàng'].fillna('Unknown')

In [18]:
# 7.xử lý cột birthdate để tính tuổi
df['Birthday'] = df['Birthday'].replace('2025-01-01', None)
df['Birthday'] = df['Birthday'].fillna('Unknown')
def calculate_age(birthday_str):
    if birthday_str == 'Unknown' or pd.isna(birthday_str):
        return None
    try:
        # Parse the birthday string to datetime
        birth_date = pd.to_datetime(birthday_str, errors='coerce')
        if pd.isna(birth_date):
            return None
        # Calculate age as 2019 minus the birth year
        age = 2019 - birth_date.year
        return age
    except:
        return None

# Apply the age calculation to the Birthday column
df['Age'] = df['Birthday'].apply(calculate_age)
df['Age'] = df['Age'].fillna('Unknown')

In [19]:
# 8. Mã hóa các cột phân loại
from sklearn.preprocessing import LabelEncoder

# Tạo encoder
le = LabelEncoder()

# Mã hóa và lưu vào cột mới
df['Trạng thái_encoded'] = le.fit_transform(df['Trạng thái'])


In [20]:
# 9.Phát hiện và đánh dấu giá trị ngoại lệ
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    print(f"Số lượng ngoại lệ ở cột {column}: {outliers.shape[0]}")
    return outliers

# Áp dụng cho các cột cần kiểm
cols = ['Salary', 'TS_CREDIT_SCORE_V2', 'NumberOfLoans', 'LongestOverdue']
for col in cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
for col in cols:
    detect_outliers_iqr(df, col)


Số lượng ngoại lệ ở cột Salary: 169
Số lượng ngoại lệ ở cột TS_CREDIT_SCORE_V2: 51
Số lượng ngoại lệ ở cột NumberOfLoans: 1
Số lượng ngoại lệ ở cột LongestOverdue: 199


In [21]:
def flag_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[column + '_outlier_flag'] = df[column].apply(lambda x: 1 if x < lower or x > upper else 0)
    return df

df = flag_outliers(df, 'Salary')
df = flag_outliers(df, 'LongestOverdue')

In [22]:
def classify_score(score):
    if score >= 750:
        return 'Rất Tốt'
    elif score >= 600:
        return 'Tốt'
    elif score >= 400:
        return 'TB'
    else:
        return 'Nguy cơ'

df['TS_CREDIT_SCORE_V2_class'] = df['TS_CREDIT_SCORE_V2'].apply(classify_score)


In [23]:
df.to_excel('/content/ggdrive/MyDrive/Phân tích hiệu suất kinh doanh TIMA/test1.xlsx', index=False)

In [24]:
df.to_csv('/content/ggdrive/MyDrive/Phân tích hiệu suất kinh doanh TIMA/test1.csv', index=False)